In [ ]:
# importing the required libraries
import pandas as pd
import numpy as np

In [ ]:
# Loading the dataset:
df = pd.read_csv('/content/laptop.csv')

In [ ]:
df.isnull().sum()

Unnamed: 0.1         0
Unnamed: 0          30
Company             30
TypeName            30
Inches              30
ScreenResolution    30
Cpu                 30
Ram                 30
Memory              30
Gpu                 30
OpSys               30
Weight              30
Price               30
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.drop(columns=['Unnamed: 0','Inches','Weight','OpSys'],inplace=True)

In [ ]:
df.isna().sum()

Unnamed: 0.1        0
Company             0
TypeName            0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
Price               0
dtype: int64

In [ ]:
# Precrocessing the dataset :
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
catgorical_columns=['Company','TypeName', 'ScreenResolution', 'Cpu', 'Gpu']
label_column1=['Ram']
label_column2=['Memory']
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(handle_unknown='ignore'),catgorical_columns)],remainder='passthrough')

In [ ]:
l=LabelEncoder()
df['Ram']=l.fit_transform(df['Ram'])
df['Memory']=l.fit_transform(df['Memory'])

In [ ]:
X=df.drop('Price',axis=1)
y=df['Price']

In [ ]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

***XGboost***

In [ ]:
# creating pipeline and buliding the model using the XGBRegressor:
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
pipe1=Pipeline(steps=[('preprocessor',ct),('model',XGBRegressor())])

In [ ]:
# Hyparameter Tuning For XGboost:
from random import random
from scipy.stats import randint, uniform
import joblib
param_dist = {
    'model__n_estimators': randint(100, 300),
    'model__learning_rate': uniform(0.01, 0.1),
    'model__max_depth': randint(3, 7),
    'model__subsample': uniform(0.8, 0.2),
    'model__colsample_bytree': uniform(0.8, 0.2)
}

# Initializing Randomized Search
random_search = RandomizedSearchCV(pipe1, param_distributions=param_dist, n_iter=50, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2, random_state=0)

# Fitting Randomized Search
random_search.fit(X_train, y_train)

# Getting best parameters
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")

# Getting best model
best_model = random_search.best_estimator_

# Predicting using the best model
y_pred1= best_model.predict(X_test)

# Saving the best model
joblib.dump(best_model, 'best_model.pkl')

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters found: {'model__colsample_bytree': 0.8894250757235256, 'model__learning_rate': 0.09464086724711278, 'model__max_depth': 4, 'model__n_estimators': 239, 'model__subsample': 0.8594873901710268}


['best_model.pkl']

In [ ]:
# evaluating the model by usieng metrics:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test,y_pred1)
rmse=np.sqrt(mean_squared_error(y_test,y_pred1))
from sklearn.model_selection import cross_val_score
score=cross_val_score(pipe1,X,y,cv=5)
from sklearn.metrics import r2_score
print(f'Score_cross_val : {score.mean()}')
print(f'mean_squared_error : {mse}')
print(f'root_mean_squared_error : {rmse}')
print(f'R2_Score is : {r2_score(y_test,y_pred1)}')

Score_cross_val : 0.7559688104867788
mean_squared_error : 263647266.72435495
root_mean_squared_error : 16237.21856490067
R2_Score is : 0.8457264961706711


***Answers of Questions ***

In [ ]:
inputs=('Lenovo','Notebook',17.3,'1600x900','Intel Core i5 8250U 1.6GHz',8,256,'Nvidia GeForce MX150','No OS','2.8')

In [ ]:
input_data = pd.DataFrame([inputs], columns=X.columns)

In [ ]:
#Does the brand of the laptop significantly influence its price
df.groupby('Company')['Price'].mean()
from scipy.stats import ttest_ind
a=ttest_ind(df[df['Company']=='Dell']['Price'],df[df['Company']=='HP']['Price'])
a.pvalue

0.04216442475420708

In [ ]:
level_of_significance=0.05
if a.pvalue<level_of_significance:
  print('Reject the null hypothesis is influences its Price')
else:
  print('Accept the null hypothesis is influences its Price')

Reject the null hypothesis is influences its Price


***both the models are giving same results.***

Questions to Explore:


1.Which features have the most significant impact on laptop prices?

Answer : Company,screen Resolution,Cpu,Ram,Memory

2.Can the model accurately predict the prices of laptops from lesser-known brands?

Answer: the Accurately predicting as in its range.

3.Does the brand of the laptop significantly influence its price?

Answer: Brand  not Influence its price.

4.How well does the model perform on laptops with high-end specifications compared to budget laptops?

Answer: The model performs well and showing the significate varience.

5.What are the limitations and challenges in predicting laptop prices accurately?

Answer: It is not accuratly predicting all the prices.

6.How does the model perform when predicting the prices of newly released laptops not present in the training dataset?

Answer Its Predicting as its in his Range.

***RandomFroest:***

In [ ]:
# Creating the pipeline and buliting the model by RandomForestRegressor:
from os import pipe
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
pipe=Pipeline(steps=[('preprocessor',ct),('model',RandomForestRegressor())])

In [ ]:
# hyparameter Tuning:
import joblib
param_dist = {
    'model__n_estimators': randint(100, 300),
    'model__max_depth': randint(3, 10),
    'model__min_samples_split': randint(2, 20),
    'model__min_samples_leaf': randint(1, 20),
    'model__max_features': ['auto', 'sqrt', 'log2'],
    'model__bootstrap': [True, False]
}

# Initializing Randomized Search
random_search = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=50, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2, random_state=0)

# Fitting Randomized Search
random_search.fit(X_train, y_train)

# Getting best parameters
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")

# Getting best model
best_model = random_search.best_estimator_

# Predicting using the best model
y_pred = best_model.predict(X_test)

# Saving the best model
joblib.dump(best_model, 'best_model_random_forest.pkl')

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Best parameters found: {'model__bootstrap': True, 'model__max_depth': 8, 'model__max_features': 'auto', 'model__min_samples_leaf': 4, 'model__min_samples_split': 5, 'model__n_estimators': 203}


['best_model_random_forest.pkl']

In [ ]:
# evaluating the model by usieng metrics:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)
from sklearn.model_selection import cross_val_score
score=cross_val_score(pipe,X,y,cv=5)
from sklearn.metrics import r2_score
mse=mean_squared_error(y_test,y_pred)
rmse=np.sqrt(mse)
print(f'mean_squared_error: {mse}')
print(f'root_mean_squared_error: {rmse}')
print(f'Score_cross_val: {score.mean()}')
print(f'R2_score : {r2_score(y_test,y_pred)}')

mean_squared_error: 457663442.70634615
root_mean_squared_error: 21393.06996918269
Score_cross_val: 0.714197484368175
R2_score : 0.7321977058282203
